In [10]:
import scripts.cccco as cccco
import scripts.college_scorecard as scorecard
import scripts.labor_market as labor_market
import scripts.join_tools as join_tools

In [11]:
import pandas as pd

## 1. Gather all relevant metadata on California Community Colleges using `colleges` endpoint of the [the CCC API](https://api.cccco.edu/)

In [12]:
all_colleges = cccco.get_ccc_colleges()

In [13]:
print(f"{all_colleges.shape[0]:,} rows")
all_colleges.head()

115 rows


,CollegeID,CollegeName,DistrictID,StreetAddress,City,County,Zip,ZipPlus4,MailingAddress,MailingCity,MailingZip,Phone,WebsiteURL,Latitude,Longitude,LogoURL,District
0,021,Cuyamaca College,020,900 Rancho San Diego Parkway,El Cajon,San Diego,92019,4304,900 Rancho San Diego Parkway,El Cajon,92019,619.660.4000,www.cuyamaca.edu,32.744890,-116.935229,CuyamacaCollegeLogo.jpg,None
1,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,92020,619.644.7000,www.grossmont.edu,32.817897,-117.005640,GrossmontCollegelogo.jpg,None
2,031,Imperial Valley College,030,380 East Aten Road,Imperial,Imperial,92251,9787,380 East Aten Road,Imperial,92251,760.352.8320,www.imperial.edu,32.825859,-115.502999,ImperialValleyCollegeLogocopy.jpg,None
3,051,MiraCosta College,050,1 Barnard Drive,Oceanside,San Diego,92056,3899,1 Barnard Drive,Oceanside,92056,760.757.2121,www.miracosta.edu,33.188864,-117.301064,Mira_Costa_College_Logo_4c.png,None
4,061,Palomar College,060,1140 West Mission Road,San Marcos,San Diego,92069,1487,1140 West Mission Road,San Marcos,92069,760.744.1150,www.palomar.edu,33.147015,-117.183980,PalomarCollegeLogo.jpg,None


## 2. Determine the top 10 most popular programs of study using [the CCC DataMart](https://datamart.cccco.edu/)

In [14]:
# Placeholder until DataMart data is available
programs = [
    "Database",
    "Computer Science",
    "Data Science",
]

## 3. Gather all colleges in the CCC system that offer a degree or certificate in those programs using `programs` endpoint [the CCC API](https://api.cccco.edu/)

In [15]:
cccco_programs = pd.DataFrame()

for program in programs:
    df = cccco.get_ccc_programs(program)
    df["Program Query"] = program
    cccco_programs = pd.concat([cccco_programs, df])

In [16]:
print(f"{cccco_programs.shape[0]:,} rows")
cccco_programs.sample(10)

207 rows


,CollegeID,CollegeName,ProgramAward,CreditType,Title,TopCode,Program Query
110,371,Cañada College,B,C,Computer Science - Java,070600,Computer Science
11,361,City College of San Francisco,M,C,Database Administration,070720,Database
112,371,Cañada College,B,C,Computer Science - Swift,070600,Computer Science
95,344,Merritt College,S,C,Computer Science,070700,Computer Science
12,233,Sacramento City College,S,C,Data Science,070700,Data Science
114,742,Los Angeles Harbor College,S,C,Computer Science and Engineering,070710,Computer Science
2,551,San Joaquin Delta College,C,C,Computer Science,070200,Computer Science
4,551,San Joaquin Delta College,S,C,Computer Science,070200,Computer Science
26,873,Santiago Canyon College,S,C,"Data Science, AS",170100,Data Science
14,181,College of the Siskiyous,S,C,Computer Science,070600,Computer Science


In [17]:
cccco_programs["Program Query"].value_counts()

Program Query
Computer Science    153
Data Science         28
Database             26
Name: count, dtype: int64

## 4. Merge the `colleges` endpoint data with the `programs` endpoint data (one row per college per program)

In [18]:
# Merge College Metadata with Program Data
# One row per college per program

college_data = pd.merge(
    all_colleges,
    cccco_programs.drop(columns=["CollegeName"]),
    on="CollegeID",
    how="right",
)

college_data.sort_values(by=["CollegeID", "Program Query"], inplace=True)

print(f"{college_data.shape[0]:,} rows")
college_data.head()

207 rows


,CollegeID,CollegeName,DistrictID,StreetAddress,City,County,Zip,ZipPlus4,MailingAddress,MailingCity,...,WebsiteURL,Latitude,Longitude,LogoURL,District,ProgramAward,CreditType,Title,TopCode,Program Query
71,021,Cuyamaca College,020,900 Rancho San Diego Parkway,El Cajon,San Diego,92019,4304,900 Rancho San Diego Parkway,El Cajon,...,www.cuyamaca.edu,32.744890,-116.935229,CuyamacaCollegeLogo.jpg,None,X,C,Computer Science,070600,Computer Science
72,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,www.grossmont.edu,32.817897,-117.005640,GrossmontCollegelogo.jpg,None,X,C,Computer Science,070600,Computer Science
178,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,www.grossmont.edu,32.817897,-117.005640,GrossmontCollegelogo.jpg,None,S,C,University Studies: Mathematics and Natural Sc...,490200,Computer Science
190,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,www.grossmont.edu,32.817897,-117.005640,GrossmontCollegelogo.jpg,None,N,C,Data Science,070700,Data Science
73,031,Imperial Valley College,030,380 East Aten Road,Imperial,Imperial,92251,9787,380 East Aten Road,Imperial,...,www.imperial.edu,32.825859,-115.502999,ImperialValleyCollegeLogocopy.jpg,None,X,C,Computer Science,070600,Computer Science


## 5. Merge the college data with the DataMart program data (one row per college per program)

In [19]:
pass # Not implemented yet

## 6. Merge the data with external sources

### Labor Market Projection Data

The issue here is that labor market data is indexed by **Standard Occupation Classification** (SOC) Codes, while college programs are indexed by **Taxonomy of Program** (TOP) Codes and by **Classification of Instructional Programs** (CIP) Codes.  The match between these is not one-to-one.

TOP Codes do not appear to be listed in .csv or .xlsx form anywhere I can think to look, but they are indirectly present in the TOP-CIP crosswalk (`./resources/TOPCIPJune26.xlsx`). Active TOP Codes can also be mined from DataMart.

Annoyingly, there is not crosswalk between TOP Codes and SOC Codes, so we'll have to map from TOP to CIP to SOC. Alternatively, we can try fuzzy joins between TOP Code Descriptions (scraped from DataMart) and SOC Code Descriptions (`./resources/soc_2018_definitions.xlsx`).

First, add the Metropolitan Statistical Area to which the college belongs to the college dataset:

In [20]:
# Gather all Metropolitan Statistical Area (MSA) data. This will allow us to
# join labor market data to the college data.

msa = labor_market.get_labor_market_msa_data()
print(f"{msa.shape[0]:,} rows")
msa.head()

29 rows


,Area Name,Metropolitan Statistical Area,MSA Counties
0,Redding MSA (Shasta County),Redding MSA,Shasta County
1,Oakland-Hayward-Berkeley MD (Alameda and Contr...,Oakland-Hayward-Berkeley MD,Alameda and Contra Costa Counties
2,Fresno MSA (Fresno County),Fresno MSA,Fresno County
3,San Jose-Sunnyvale-Santa Clara MSA (San Benito...,San Jose-Sunnyvale-Santa Clara MSA,San Benito and Santa Clara Counties
4,Riverside-San Bernardino-Ontario MSA (Riversid...,Riverside-San Bernardino-Ontario MSA,Riverside and San Bernardino Counties


In [21]:
# Add MSA data to `all_colleges` dataset
area_counties = msa["MSA Counties"].dropna().unique()

all_colleges["Matched Counties"] = all_colleges["County"].apply(
    lambda x: join_tools.get_best_match(x, area_counties)
)

# One row per college per program
all_colleges = pd.merge(
    all_colleges,
    msa,
    left_on="Matched Counties",
    right_on="MSA Counties",
    how="left"
)

print(f"{all_colleges.shape[0]:,} rows")
all_colleges.head()

115 rows


,CollegeID,CollegeName,DistrictID,StreetAddress,City,County,Zip,ZipPlus4,MailingAddress,MailingCity,...,Phone,WebsiteURL,Latitude,Longitude,LogoURL,District,Matched Counties,Area Name,Metropolitan Statistical Area,MSA Counties
0,021,Cuyamaca College,020,900 Rancho San Diego Parkway,El Cajon,San Diego,92019,4304,900 Rancho San Diego Parkway,El Cajon,...,619.660.4000,www.cuyamaca.edu,32.744890,-116.935229,CuyamacaCollegeLogo.jpg,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
1,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,619.644.7000,www.grossmont.edu,32.817897,-117.005640,GrossmontCollegelogo.jpg,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
2,031,Imperial Valley College,030,380 East Aten Road,Imperial,Imperial,92251,9787,380 East Aten Road,Imperial,...,760.352.8320,www.imperial.edu,32.825859,-115.502999,ImperialValleyCollegeLogocopy.jpg,None,Imperial County,El Centro MSA (Imperial County),El Centro MSA,Imperial County
3,051,MiraCosta College,050,1 Barnard Drive,Oceanside,San Diego,92056,3899,1 Barnard Drive,Oceanside,...,760.757.2121,www.miracosta.edu,33.188864,-117.301064,Mira_Costa_College_Logo_4c.png,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
4,061,Palomar College,060,1140 West Mission Road,San Marcos,San Diego,92069,1487,1140 West Mission Road,San Marcos,...,760.744.1150,www.palomar.edu,33.147015,-117.183980,PalomarCollegeLogo.jpg,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County


In [22]:
# Add MSA data to `college_data` dataset
area_counties = msa["MSA Counties"].dropna().unique()

college_data["Matched Counties"] = college_data["County"].apply(
    lambda x: join_tools.get_best_match(x, area_counties)
)

# One row per college per program
college_data = pd.merge(
    college_data,
    msa,
    left_on="Matched Counties",
    right_on="MSA Counties",
    how="left"
)

print(f"{college_data.shape[0]:,} rows")
college_data.head()

207 rows


,CollegeID,CollegeName,DistrictID,StreetAddress,City,County,Zip,ZipPlus4,MailingAddress,MailingCity,...,District,ProgramAward,CreditType,Title,TopCode,Program Query,Matched Counties,Area Name,Metropolitan Statistical Area,MSA Counties
0,021,Cuyamaca College,020,900 Rancho San Diego Parkway,El Cajon,San Diego,92019,4304,900 Rancho San Diego Parkway,El Cajon,...,None,X,C,Computer Science,070600,Computer Science,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
1,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,None,X,C,Computer Science,070600,Computer Science,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
2,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,None,S,C,University Studies: Mathematics and Natural Sc...,490200,Computer Science,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
3,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,None,N,C,Data Science,070700,Data Science,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
4,031,Imperial Valley College,030,380 East Aten Road,Imperial,Imperial,92251,9787,380 East Aten Road,Imperial,...,None,X,C,Computer Science,070600,Computer Science,Imperial County,El Centro MSA (Imperial County),El Centro MSA,Imperial County


Next, we can join relevant labor market data by matching the TOP Code from the college data set to the SOC Code from the labor market data set. Since this is a one-to-many match, we won't do this at the DataFrame level.

Instead, we can perhaps consider the following use case:

1. User selects a job of interest (e.g., "Data Science").
2. We locate the relevant SOC Code data from labor market data set.
3. Rank MSA's by growth in that field (by percent growth or numeric?)
4. Report the colleges in the MSA with the top growth and add programs with corresponding TOP Codes.

In [23]:
# Step 1, 2, and 3:
occupation = "Computer" #"Statistician"
occupation_by_msa = labor_market.get_occupation_projections_by_title(occupation)
print(f"{occupation_by_msa.shape[0]:,} rows")
occupation_by_msa.head()

320 rows


,_id,_full_text,Area Type,Area Name,Period,SOC Level,Standard Occupational Classification (SOC),Occupational Title,Base Year Employment Estimate,Projected Year Employment Estimate,Numeric Change,Percentage Change,Exits,Transfers,Total Job Openings,Median Hourly Wage,Median Annual Wage,Entry Level Education,Work Experience,Job Training
0,2040,'-2030':8 '-3021':11 '11':10 '110':17 '136088'...,Metropolitan Area,Chico MSA (Butte County),2020-2030,4,11-3021,Computer and Information Systems Managers,110.0,160.0,50.0,45.5,20.0,80.0,150.0,65.42,136088.0,Bachelor's degree,5 years or more,None
1,3804,"'-2030':10 '-3021':13 '10':21,23 '106152':27 '...",Metropolitan Area,Hanford-Corcoran MSA (Kings County),2020-2030,4,11-3021,Computer and Information Systems Managers,30.0,40.0,10.0,33.3,10.0,20.0,40.0,51.04,106152.0,Bachelor's degree,5 years or more,None
2,5646,'-2030':8 '-3021':11 '11':10 '120':22 '136026'...,Metropolitan Area,Modesto MSA (Stanislaus County),2020-2030,4,11-3021,Computer and Information Systems Managers,180.0,230.0,50.0,27.8,30.0,120.0,200.0,65.40,136026.0,Bachelor's degree,5 years or more,None
3,15694,'-2030':10 '-3021':13 '11':12 '120':24 '155953...,Metropolitan Area,Vallejo-Fairfield MSA (Solano County),2020-2030,4,11-3021,Computer and Information Systems Managers,190.0,240.0,50.0,26.3,40.0,120.0,210.0,74.98,155953.0,Bachelor's degree,5 years or more,None
4,11604,'-2030':20 '-3021':23 '102.84':36 '11':22 '125...,Metropolitan Area,San Francisco-Redwood City-South San Francisco...,2020-2030,4,11-3021,Computer and Information Systems Managers,19660.0,24660.0,5000.0,25.4,3600.0,12590.0,21190.0,102.84,213890.0,Bachelor's degree,5 years or more,None


In [24]:
# Get the MSA with the highest employment percent change for each occupation
max_msa_for_occupation = (
    occupation_by_msa.groupby("Standard Occupational Classification (SOC)")
    .first()
    .reset_index()
)

max_msa_for_occupation

,Standard Occupational Classification (SOC),_id,_full_text,Area Type,Area Name,Period,SOC Level,Occupational Title,Base Year Employment Estimate,Projected Year Employment Estimate,Numeric Change,Percentage Change,Exits,Transfers,Total Job Openings,Median Hourly Wage,Median Annual Wage,Entry Level Education,Work Experience,Job Training
0,11-3021,2040,'-2030':8 '-3021':11 '11':10 '110':17 '136088'...,Metropolitan Area,Chico MSA (Butte County),2020-2030,4,Computer and Information Systems Managers,110.0,160.0,50.0,45.5,20.0,80.0,150.0,65.42,136088.0,Bachelor's degree,5 years or more,None
1,15-1211,3838,"'-1211':13 '-2030':10 '10':19,21 '15':12 '20':...",Metropolitan Area,Hanford-Corcoran MSA (Kings County),2020-2030,4,Computer Systems Analysts,30.0,40.0,10.0,33.3,10.0,20.0,40.0,47.05,97880.0,Bachelor's degree,None,None
2,15-1221,11659,'-1221':23 '-2030':20 '1260':34 '15':22 '2000'...,Metropolitan Area,San Francisco-Redwood City-South San Francisco...,2020-2030,4,Computer and Information Research Scientists,2000.0,2980.0,980.0,49.0,440.0,1260.0,2680.0,98.08,204011.0,Master's degree,None,None
3,15-1231,2083,'-1231':11 '-2030':8 '10':20 '15':10 '20':18 '...,Metropolitan Area,Chico MSA (Butte County),2020-2030,4,Computer Network Support Specialists,60.0,80.0,20.0,33.3,10.0,40.0,70.0,30.54,63527.0,Associate's degree,None,None
4,15-1232,5689,'-1232':11 '-2030':8 '15':10 '170':21 '2020':7...,Metropolitan Area,Modesto MSA (Stanislaus County),2020-2030,4,Computer User Support Specialists,280.0,350.0,70.0,25.0,60.0,170.0,300.0,29.82,62014.0,"Some college, no degree",None,None
5,15-1241,10504,"'-1241':11 '-2030':8 '10':17,19 '101258':23 '1...",Metropolitan Area,Salinas MSA (Monterey County),2020-2030,4,Computer Network Architects,50.0,60.0,10.0,20.0,10.0,30.0,50.0,48.68,101258.0,Bachelor's degree,5 years or more,None
6,15-1244,2887,"'-1244':12 '-2030':9 '10':20,22 '100025':26 '1...",Metropolitan Area,El Centro MSA (Imperial County),2020-2030,4,Network and Computer Systems Administrators,40.0,50.0,10.0,25.0,10.0,20.0,40.0,48.09,100025.0,Bachelor's degree,None,None
7,15-1251,11665,'-1251':23 '-2030':20 '124906':34 '15':22 '158...,Metropolitan Area,San Francisco-Redwood City-South San Francisco...,2020-2030,4,Computer Programmers,3310.0,3600.0,290.0,8.8,690.0,1580.0,2560.0,60.05,124906.0,Bachelor's degree,None,None
8,15-1299,2090,'-1299':11 '-2030':8 '100':21 '15':10 '180':16...,Metropolitan Area,Chico MSA (Butte County),2020-2030,4,"Computer Occupations, All Other",180.0,230.0,50.0,27.8,50.0,100.0,200.0,36.75,76435.0,Bachelor's degree,None,None
9,17-2061,11683,'-2030':20 '-2061':23 '1380':32 '17':22 '19868...,Metropolitan Area,San Francisco-Redwood City-South San Francisco...,2020-2030,4,Computer Hardware Engineers,2570.0,3320.0,750.0,29.2,560.0,1380.0,2690.0,95.52,198681.0,Bachelor's degree,None,None


In [25]:
# Step 4, part 1: Gather all colleges in this MSA

max_msa_colleges = pd.merge(
    all_colleges,
    max_msa_for_occupation,
    left_on="Area Name",
    right_on="Area Name",
    how="inner",
).sort_values(
    by=[
        "Standard Occupational Classification (SOC)",
        "Metropolitan Statistical Area",
        "CollegeName",
    ]
).reset_index(drop=True)

# Print relevant information:
max_msa_colleges[
    [
        "CollegeID",
        "CollegeName",
        "Matched Counties",
        "Area Name",
        "Metropolitan Statistical Area",
        "MSA Counties",
        "Standard Occupational Classification (SOC)",
        "Occupational Title",
    ]
]

,CollegeID,CollegeName,Matched Counties,Area Name,Metropolitan Statistical Area,MSA Counties,Standard Occupational Classification (SOC),Occupational Title
0,111,Butte College,Butte County,Chico MSA (Butte County),Chico MSA,Butte County,11-3021,Computer and Information Systems Managers
1,582,Lemoore College,Kings County,Hanford-Corcoran MSA (Kings County),Hanford-Corcoran MSA,Kings County,15-1211,Computer Systems Analysts
2,371,Cañada College,San Francisco and San Mateo Counties,San Francisco-Redwood City-South San Francisco...,San Francisco-Redwood City-South San Francisco MD,San Francisco and San Mateo Counties,15-1221,Computer and Information Research Scientists
3,361,City College of San Francisco,San Francisco and San Mateo Counties,San Francisco-Redwood City-South San Francisco...,San Francisco-Redwood City-South San Francisco MD,San Francisco and San Mateo Counties,15-1221,Computer and Information Research Scientists
4,372,College of San Mateo,San Francisco and San Mateo Counties,San Francisco-Redwood City-South San Francisco...,San Francisco-Redwood City-South San Francisco MD,San Francisco and San Mateo Counties,15-1221,Computer and Information Research Scientists
5,373,Skyline College,San Francisco and San Mateo Counties,San Francisco-Redwood City-South San Francisco...,San Francisco-Redwood City-South San Francisco MD,San Francisco and San Mateo Counties,15-1221,Computer and Information Research Scientists
6,111,Butte College,Butte County,Chico MSA (Butte County),Chico MSA,Butte County,15-1231,Computer Network Support Specialists
7,121,Feather River College,Stanislaus County,Modesto MSA (Stanislaus County),Modesto MSA,Stanislaus County,15-1232,Computer User Support Specialists
8,592,Modesto Junior College,Stanislaus County,Modesto MSA (Stanislaus County),Modesto MSA,Stanislaus County,15-1232,Computer User Support Specialists
9,451,Hartnell College,Monterey County,Salinas MSA (Monterey County),Salinas MSA,Monterey County,15-1241,Computer Network Architects


In [26]:
# Now, gather the SOC Codes from the labor market data
max_soc_codes = (
    max_msa_colleges["Standard Occupational Classification (SOC)"].unique()
)

max_soc_codes

array(['11-3021', '15-1211', '15-1221', '15-1231', '15-1232', '15-1241',
       '15-1244', '15-1251', '15-1299', '17-2061', '17-2072', '25-1021',
       '43-9071', '49-2011', '51-9161', '51-9162'], dtype=object)

In [29]:
# Build a DataFrame of colleges that offer programs in the relevant TOP Codes
# for each of the top SOC Codes

matched_top_codes = pd.DataFrame()
for soc in max_soc_codes:
    df = join_tools.match_colleges_top_from_soc(soc)
    df["SocCode"] = soc
    df["SocTitle"] = occupation_by_msa.loc[
        occupation_by_msa["Standard Occupational Classification (SOC)"] == soc,
        "Occupational Title"
    ].values[0]
    matched_top_codes = pd.concat([matched_top_codes, df])

print(f"{matched_top_codes.shape[0]:,} rows")
matched_top_codes.head()

Error thrown when attempting SOC-TOP Merge with soc = '17-2072':
KeyError 'TopCode'
Error thrown when attempting SOC-TOP Merge with soc = '43-9071':
KeyError 'TopCode'
Error thrown when attempting SOC-TOP Merge with soc = '51-9161':
KeyError 'TopCode'
Error thrown when attempting SOC-TOP Merge with soc = '51-9162':
KeyError 'TopCode'
9,093 rows


,CollegeID,CollegeName,ProgramAward,CreditType,Title,TopCode,TopCodeTitle,SocCode,SocTitle
0,021,Cuyamaca College,X,C,Computer Science,070600,Computer Science (Transfer),11-3021,Computer and Information Systems Managers
1,021,Cuyamaca College,S,C,"Networking, Security and System Administration...",070810,Computer Networking,11-3021,Computer and Information Systems Managers
2,021,Cuyamaca College,T,C,"Networking, Security and System Administration...",070810,Computer Networking,11-3021,Computer and Information Systems Managers
3,021,Cuyamaca College,S,C,"Networking, Security and System Administration...",070810,Computer Networking,11-3021,Computer and Information Systems Managers
4,021,Cuyamaca College,T,C,"Networking, Security and System Administration...",070810,Computer Networking,11-3021,Computer and Information Systems Managers


### College Scorecard data

In [ ]:
pass # Not implemented yet

### Rate My Professors NLP data

In [ ]:
pass # Not implemented yet